In [1]:
from PepitoDataset import PepitoDataset
from Models import PepitoModel
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np

# Data

In [2]:
# Load the dataset
dataset = PepitoDataset("./dataset")

# Split the dataset into train and test and create the dataloaders
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

# Model

In [3]:
for data in train_dataloader:
    inputs, labels = data
    print(type(inputs))
    print(labels)


    break

<class 'torch.Tensor'>
tensor([1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0])


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

# Assuming PepitoModel, train_dataloader, and test_dataloader are already defined

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PepitoModel(2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


def train():
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f"[{i + 1}, {running_loss / 2000:.3f}]")
            running_loss = 0.0


def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_dataloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}")
    return accuracy


for epoch in range(10):
    train()
    test()
    torch.save(model.state_dict(), f"model_{epoch}.pt")

/home/tsauv/Dev/pepito/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tsauv/Dev/pepito/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Accuracy: 38.83
Accuracy: 61.17
Accuracy: 61.17
Accuracy: 61.17
Accuracy: 61.17
Accuracy: 61.17
Accuracy: 61.17
Accuracy: 61.17
Accuracy: 61.17
Accuracy: 61.17
